Imports first, then load the raw data:

In [1]:
import re
import os
import pickle
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
money=pickle.load(open("../data/raw/salaries.pickle", "rb"))
advs=pickle.load(open("../data/raw/advstats.pickle", "rb"))
stats=pickle.load(open("../data/raw/regstats.pickle", "rb"))
rookies=pickle.load(open("../data/raw/rookies.pickle", "rb"))

# Salary List Data

Quickly look at one of the dataframes:

In [3]:
money[2018].apply(lambda x: x.head(5).append(x.tail(5))) #look at first and last five

,0,1,2,3
0,RK,NAME,TEAM,SALARY
1,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154"
2,2,"Russell Westbrook, PG",Oklahoma City Thunder,"$35,654,150"
3,3,"Chris Paul, PG",Houston Rockets,"$35,654,150"
4,4,"Blake Griffin, PF",Detroit Pistons,"$32,088,932"
477,434,"Jonathan Gibson, PG",Boston Celtics,"$17,092"
478,435,"Tarik Phillip, G",Washington Wizards,"$9,474"
479,436,"Duncan Robinson, SF",Miami Heat,"$9,474"
480,437,"Theo Pinson, SG",Brooklyn Nets,"$4,737"
481,438,"Kendrick Nunn, SG",Miami Heat,"$4,737"


In [4]:
money.keys()

dict_keys([2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009])

In [5]:
money[2017].columns

Int64Index([0, 1, 2, 3], dtype='int64')

Certain things we need to fix: 

1. Fix some headers (0 -> Rank, 1-> Name, 2-> Team, 3-> Salary)
2. Remove rows with those header labels, as they were repeated in the website tables
3. Add year column for when the lists are aggregated into a single dataframe

This we can do after aggregations:

1. Change Salary format (remove $ and commas)
2. Split position from the name into a new column
3. Change Salary datatype to int
4. Remove Rk column; it's not significant

In [6]:
import re
combined ={}

for k,v in money.items():
    v.columns = ['Rk','Player','Tm','Salary']
    v = v[v.Rk!= "RK"]
    v["Year"] = k
    combined[k]=v

/Users/youngjeong/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [7]:
combined[2017].head(10)

,Rk,Player,Tm,Salary,Year
1,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154",2017
2,2,"Blake Griffin, PF",LA Clippers,"$32,088,932",2017
3,3,"Paul Millsap, PF",Denver Nuggets,"$31,269,231",2017
4,4,"Kyle Lowry, PG",Toronto Raptors,"$31,200,000",2017
5,5,"Gordon Hayward, SF",Boston Celtics,"$29,727,900",2017
6,6,"Mike Conley, PG",Memphis Grizzlies,"$28,530,608",2017
7,7,"Russell Westbrook, PG",Oklahoma City Thunder,"$28,530,608",2017
8,8,"James Harden, PG",Houston Rockets,"$28,299,399",2017
9,9,"DeMar DeRozan, SG",Toronto Raptors,"$27,739,975",2017
10,10,"Al Horford, C",Boston Celtics,"$27,734,406",2017


Now we can combine all the dataframes into a single one and get the FA information from 2011-2018 (2018 Salary information will be our test_y)

In [8]:
from functools import reduce
salaries = reduce(lambda x,y:pd.concat([x,y]),[v for k,v in combined.items()])

In [9]:
salaries.shape

(4511, 5)

In [10]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4511 entries, 1 to 463
Data columns (total 5 columns):
Rk        4511 non-null object
Player    4511 non-null object
Tm        4511 non-null object
Salary    4511 non-null object
Year      4511 non-null int64
dtypes: int64(1), object(4)
memory usage: 211.5+ KB


In [11]:
salaries.head(5)

,Rk,Player,Tm,Salary,Year
1,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154",2018
2,2,"Russell Westbrook, PG",Oklahoma City Thunder,"$35,654,150",2018
3,3,"Chris Paul, PG",Houston Rockets,"$35,654,150",2018
4,4,"Blake Griffin, PF",Detroit Pistons,"$32,088,932",2018
5,5,"Gordon Hayward, SF",Boston Celtics,"$31,214,295",2018


Now we can changed the Salary format to remove $ and commas, as well as splitting 

In [12]:
salaries["Salary"] = salaries["Salary"].str.replace('$','').str.replace(',','')

In [13]:
salaries.Salary = salaries.Salary.astype(int)
del salaries['Rk']
salaries['Player'], salaries['Pos'] = salaries['Player'].str.split(', ', 1).str

In [14]:
salaries.head(5)

,Player,Tm,Salary,Year,Pos
1,Stephen Curry,Golden State Warriors,37457154,2018,PG
2,Russell Westbrook,Oklahoma City Thunder,35654150,2018,PG
3,Chris Paul,Houston Rockets,35654150,2018,PG
4,Blake Griffin,Detroit Pistons,32088932,2018,PF
5,Gordon Hayward,Boston Celtics,31214295,2018,SF


# Player Stats Data

Now we can clean up the stats data. First quickly look at one of the regular stats dataframe and advanced stats dataframe:

In [15]:
stats.keys()

dict_keys([2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008])

In [16]:
stats[2019].tail(10)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
724,525,Thaddeus Young,PF,30,IND,81,81,30.7,5.5,10.4,...,.644,2.4,4.1,6.5,2.5,1.5,0.4,1.5,2.4,12.6
725,526,Trae Young,PG,20,ATL,81,81,30.9,6.5,15.5,...,.829,0.8,2.9,3.7,8.1,0.9,0.2,3.8,1.7,19.1
726,527,Cody Zeller,C,26,CHO,49,47,25.4,3.9,7.0,...,.787,2.2,4.6,6.8,2.1,0.8,0.8,1.3,3.3,10.1
727,528,Tyler Zeller,C,29,TOT,6,1,15.5,2.7,5.0,...,.778,1.8,2.2,4.0,0.7,0.2,0.5,0.7,3.3,7.7
728,528,Tyler Zeller,C,29,ATL,2,0,5.5,0.0,1.0,...,NaN,1.0,2.0,3.0,0.5,0.0,0.0,0.0,2.0,0.0
729,528,Tyler Zeller,C,29,MEM,4,1,20.5,4.0,7.0,...,.778,2.3,2.3,4.5,0.8,0.3,0.8,1.0,4.0,11.5
730,529,Ante Zizic,C,22,CLE,59,25,18.3,3.1,5.6,...,.705,1.8,3.6,5.4,0.9,0.2,0.4,1.0,1.9,7.8
731,530,Ivica Zubac,C,21,TOT,59,37,17.6,3.6,6.4,...,.802,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9
732,530,Ivica Zubac,C,21,LAL,33,12,15.6,3.4,5.8,...,.864,1.6,3.3,4.9,0.8,0.1,0.8,1.0,2.2,8.5
733,530,Ivica Zubac,C,21,LAC,26,25,20.2,3.8,7.2,...,.733,2.3,5.3,7.7,1.5,0.4,0.9,1.4,2.5,9.4


In [17]:
advs[2015].head(7)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
0,1,Quincy Acy,PF,24,NYK,68,1287,11.9,.533,.181,...,NaN,1.0,0.7,1.7,.063,NaN,-2.3,-0.8,-3.1,-0.3
1,2,Jordan Adams,SG,20,MEM,30,248,12.8,.489,.291,...,NaN,0.0,0.4,0.4,.073,NaN,-1.8,1.2,-0.6,0.1
2,3,Steven Adams,C,21,OKC,70,1771,14.1,.549,.005,...,NaN,1.9,2.2,4.1,.111,NaN,-1.4,1.8,0.4,1.1
3,4,Jeff Adrien,PF,28,MIN,17,215,14.2,.494,.000,...,NaN,0.2,0.2,0.4,.087,NaN,-2.7,0.5,-2.2,0.0
4,5,Arron Afflalo,SG,29,TOT,78,2502,10.7,.533,.377,...,NaN,1.6,1.0,2.6,.050,NaN,-0.5,-1.3,-1.8,0.1
5,5,Arron Afflalo,SG,29,DEN,53,1750,11.7,.533,.370,...,NaN,1.4,0.4,1.8,.050,NaN,0.0,-1.6,-1.6,0.2
6,5,Arron Afflalo,SG,29,POR,25,752,8.2,.533,.396,...,NaN,0.2,0.5,0.8,.049,NaN,-1.5,-0.8,-2.3,-0.1


## Before Aggregation of Regular and Advanced Stats Year-wise

Some things I notice:

- Both: There are players who were traded mid-season that have appeared as rows of both teams, and a total season. I want to keep the cumulative total row (Tm assigned to TOT) and get rid of partial team stats.
- Both: It will be useful to again add a Year column for after I aggregate each lists into a single dataframe. This can be done after this step

In [18]:
for i,j in stats.items():
    temp = j 
    temp['total'] = (temp['Tm'] == 'TOT')
    temp = temp.sort_values('total', ascending=False).drop_duplicates(['Player','Age']).drop('total', 1)
    stats[i]=temp

In [19]:
for i,j in advs.items():
    temp = j 
    temp['total'] = (temp['Tm'] == 'TOT')
    temp = temp.sort_values('total', ascending=False).drop_duplicates(['Player','Age']).drop('total', 1)
    advs[i]=temp

We can now aggregate advanced stats and regular stats PER year first:

In [20]:
combined={}
for (a1,b1),(a2,b2) in zip(stats.items(),advs.items()):
    df = b1.merge(b2, how="inner",on=["Player","Age","Pos","Tm","G"])#,"MP"])
#     pd.DataFrame(sorted(df.values, key=lambda x: x[1].split(' ')[::-1]),columns=df.columns)
    combined[a1]=df.sort_values("Player")
    print("Stats Row for "+str(a1)+": "+str(b1.shape[0])
          +", Adv Row for "+str(a2)+": "+str(b2.shape[0])+", After combined: "+str(df.shape[0]))
    

Stats Row for 2019: 530, Adv Row for 2019: 530, After combined: 530
Stats Row for 2018: 540, Adv Row for 2018: 540, After combined: 540
Stats Row for 2017: 486, Adv Row for 2017: 486, After combined: 486
Stats Row for 2016: 476, Adv Row for 2016: 476, After combined: 476
Stats Row for 2015: 492, Adv Row for 2015: 492, After combined: 492
Stats Row for 2014: 482, Adv Row for 2014: 482, After combined: 482
Stats Row for 2013: 469, Adv Row for 2013: 469, After combined: 469
Stats Row for 2012: 478, Adv Row for 2012: 478, After combined: 478
Stats Row for 2011: 452, Adv Row for 2011: 452, After combined: 452
Stats Row for 2010: 442, Adv Row for 2010: 442, After combined: 442
Stats Row for 2009: 445, Adv Row for 2009: 445, After combined: 445
Stats Row for 2008: 451, Adv Row for 2008: 451, After combined: 451


## Before Aggregation of Yearly stats into a single dataframe

Now we have combined stats for each year. We can take a look at one of them:

In [21]:
combined[2017].head(10)

,Rk_x,Player,Pos,Age,Tm,G,GS,MP_x,FG,FGA,...,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
476,170,A.J. Hammons,C,24,DAL,22,0,7.4,0.8,1.9,...,NaN,-0.2,0.2,0.0,-0.001,NaN,-7.5,2.0,-5.6,-0.1
355,58,Aaron Brooks,PG,32,IND,65,0,13.8,1.9,4.6,...,NaN,-0.2,0.5,0.3,.016,NaN,-2.1,-2.5,-4.6,-0.6
454,157,Aaron Gordon,SF,21,ORL,80,72,28.7,4.9,10.8,...,NaN,2.0,1.7,3.7,.077,NaN,-0.2,-0.4,-0.7,0.8
462,181,Aaron Harrison,SG,22,CHO,5,0,3.4,0.0,0.8,...,NaN,-0.1,0.0,-0.1,-0.146,NaN,-9.6,-2.1,-11.6,0.0
83,352,Adreian Payne,PF,25,MIN,18,0,7.5,1.3,3.0,...,NaN,0.0,0.2,0.2,.086,NaN,-2.2,0.7,-1.5,0.0
396,203,Al Horford,C,30,BOS,68,68,32.3,5.6,11.8,...,NaN,3.6,2.7,6.3,.138,NaN,1.0,2.0,3.1,2.8
399,221,Al Jefferson,C,32,IND,66,1,14.1,3.6,7.1,...,NaN,1.2,1.1,2.3,.119,NaN,-1.5,-1.5,-3.1,-0.2
343,10,Al-Farouq Aminu,PF,26,POR,61,25,29.1,3.0,7.6,...,NaN,-0.1,2.0,1.9,.051,NaN,-2.3,1.2,-1.1,0.4
345,12,Alan Anderson,SF,34,LAC,30,0,10.3,1.0,2.7,...,NaN,0.0,0.1,0.1,.020,NaN,-2.6,-2.3,-4.9,-0.2
194,464,Alan Williams,C,24,PHO,47,0,15.1,2.9,5.7,...,NaN,1.1,0.9,2.1,.142,NaN,-1.8,0.1,-1.7,0.1


In [22]:
combined[2015].columns

Index(['Rk_x', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP_x', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Rk_y', 'MP_y', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
       'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'Unnamed: 19', 'OWS', 'DWS',
       'WS', 'WS/48', 'Unnamed: 24', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

Some basic cleaning we can do before we combine all of the years into one dataframe:

1. Some columns can be eliminated (Two "unnamed" arbitrary empty columns were on the website when I scraped. Rk_x and Rk_y were arbitrary rankings done by alphabetic order and is insiginficant as well)
2. We can remove one of the MP (Minutes Played) columns; there was a conflict during dataframe merge because regular stats data compiled minutes played as per game average, whereas the Advance stats data compiled minutes played as season total. I will remove MP_y.
3. Add the year of the player stat here in a column called 'Year'.

In [23]:
for k,v in combined.items():
    v=v.drop(['Rk_x','Unnamed: 19','Unnamed: 24', 'Rk_y','MP_y'], axis=1)
    v['Year'] = k
    combined[k]=v

In [24]:
combined_stats = reduce(lambda x,y:pd.concat([x,y]),[v for k,v in combined.items() if k != 2019 or k != 2008])
combined_stats = combined_stats.reset_index(drop=True);

In [25]:
combined_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5743 entries, 0 to 5742
Data columns (total 50 columns):
Player    5743 non-null object
Pos       5743 non-null object
Age       5743 non-null object
Tm        5743 non-null object
G         5743 non-null object
GS        5743 non-null object
MP_x      5743 non-null object
FG        5743 non-null object
FGA       5743 non-null object
FG%       5723 non-null object
3P        5743 non-null object
3PA       5743 non-null object
3P%       5019 non-null object
2P        5743 non-null object
2PA       5743 non-null object
2P%       5699 non-null object
eFG%      5723 non-null object
FT        5743 non-null object
FTA       5743 non-null object
FT%       5532 non-null object
ORB       5743 non-null object
DRB       5743 non-null object
TRB       5743 non-null object
AST       5743 non-null object
STL       5743 non-null object
BLK       5743 non-null object
TOV       5743 non-null object
PF        5743 non-null object
PTS       5743 non-null o

Now we can convert some of the datatypes into what we want: 

- Player, Position, Tm, Year -> unchanged
- Age, G, GS -> int
- Everything else -> floats

In [26]:
unchanged = ['Player','Pos','Tm','Year']
intlist = ['Age','G','GS']
floatlist= combined_stats.columns.difference(unchanged+intlist)

In [27]:
combined_stats[intlist] = combined_stats[intlist].astype(int)
combined_stats[floatlist] = combined_stats[floatlist].astype(float)

In [28]:
combined_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5743 entries, 0 to 5742
Data columns (total 50 columns):
Player    5743 non-null object
Pos       5743 non-null object
Age       5743 non-null int64
Tm        5743 non-null object
G         5743 non-null int64
GS        5743 non-null int64
MP_x      5743 non-null float64
FG        5743 non-null float64
FGA       5743 non-null float64
FG%       5723 non-null float64
3P        5743 non-null float64
3PA       5743 non-null float64
3P%       5019 non-null float64
2P        5743 non-null float64
2PA       5743 non-null float64
2P%       5699 non-null float64
eFG%      5723 non-null float64
FT        5743 non-null float64
FTA       5743 non-null float64
FT%       5532 non-null float64
ORB       5743 non-null float64
DRB       5743 non-null float64
TRB       5743 non-null float64
AST       5743 non-null float64
STL       5743 non-null float64
BLK       5743 non-null float64
TOV       5743 non-null float64
PF        5743 non-null float64
PTS   

In [29]:
combined_stats.head(10)

,Player,Pos,Age,Tm,G,GS,MP_x,FG,FGA,FG%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year
0,Aaron Gordon,PF,23,ORL,78,78,33.8,6.0,13.4,0.449,...,21.8,1.8,3.3,5.1,0.093,-0.2,1.2,1.0,2.0,2019
1,Aaron Holiday,PG,22,IND,50,0,12.9,2.1,5.2,0.401,...,21.9,0.1,0.8,0.9,0.065,-1.5,-1.1,-2.6,-0.1,2019
2,Abdel Nader,SF,25,OKC,61,1,11.4,1.5,3.5,0.423,...,15.1,0.0,0.9,0.9,0.062,-4.2,-0.9,-5.1,-0.5,2019
3,Al Horford,C,32,BOS,68,68,29.0,5.7,10.6,0.535,...,18.9,4.5,2.9,7.5,0.181,2.1,2.7,4.8,3.4,2019
4,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,0.433,...,13.7,3.0,2.8,5.8,0.121,0.1,0.8,0.9,1.7,2019
5,Alan Williams,PF,26,BRK,5,0,5.2,1.6,2.6,0.615,...,25.5,0.1,0.1,0.2,0.313,3.1,2.7,5.8,0.1,2019
6,Alec Burks,SG,27,TOT,64,24,21.5,3.0,7.4,0.405,...,19.0,0.7,0.8,1.5,0.052,-1.5,-0.6,-2.2,-0.1,2019
7,Alex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,...,12.2,0.1,0.6,0.6,0.053,-2.4,-0.9,-3.4,-0.2,2019
8,Alex Caruso,PG,24,LAL,25,4,21.2,3.1,6.9,0.445,...,19.0,0.5,0.6,1.0,0.093,-0.9,0.0,-0.9,0.2,2019
9,Alex Len,C,25,ATL,77,31,20.1,4.2,8.4,0.494,...,22.1,2.2,1.0,3.2,0.100,-0.1,-0.8,-0.9,0.4,2019


# Rookies list and using it to remove from player stats

Rookies have no previous year's stats (because they were in college or overseas), so we cannot use their data. Therefore we must identify the rookies for each year and remove them from the stats list.

First load the rookies pickle data:

In [30]:
rookies[2017].Player.value_counts()

Player                     4
Bryn Forbes                1
Andrew Harrison            1
Isaiah Whitehead           1
Quinn Cook                 1
Domantas Sabonis           1
Juan Hernangomez           1
Brandon Ingram             1
A.J. Hammons               1
Alex Poythress             1
Troy Williams              1
Malcolm Brogdon            1
Deyonta Davis              1
Dragan Bender              1
Ben Bentil                 1
Marcus Georges-Hunt        1
Treveon Graham             1
Patricio Garino            1
Malachi Richardson         1
Brice Johnson              1
Dejounte Murray            1
Semaj Christon             1
Georges Niang              1
Shawn Long                 1
Damian Jones               1
Dorian Finney-Smith        1
Nicolas Brussino           1
Fred VanVleet              1
Demetrius Jackson          1
Skal Labissiere            1
                          ..
Derrick Jones              1
Thon Maker                 1
Henry Ellenson             1
Mike Tobey    

There are some weird "Player" that shows up. Also some null values got picked up when it was scraped. We can remove those rows ("Player" is part of the table header that got repeated on Basketball Reference). We can also add a Year column that will be useful to identify which year the rookies belong (just like our other lists). After that we can concatenate the dataframes into a single one.

In [31]:
combined_rookies = pd.DataFrame()
for v,k in rookies.items():
    temp = rookies[v][rookies[v].Player != 'Player']
    temp = temp[~(temp.Player.isnull())]
    temp['Year']=v
    combined_rookies = pd.concat([combined_rookies,temp])
    

In [32]:
combined_rookies.head(5)

,Player,Year
0,Bam Adebayo,2018
1,Jarrett Allen,2018
2,Kadeem Allen,2018
3,Ike Anigbogu,2018
4,OG Anunoby,2018


In [33]:
combined_rookies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 851 entries, 0 to 69
Data columns (total 2 columns):
Player    851 non-null object
Year      851 non-null int64
dtypes: int64(1), object(1)
memory usage: 19.9+ KB


# Remove the rookies from the stats list

Now we have our rookies list and ready to remove the stats from those players.

In [34]:
combined_stats.shape

(5743, 50)

In [35]:
combined_rookies.shape

(851, 2)

In [36]:
COLS = ['Player','Year']

no_rookies = combined_stats.merge(combined_rookies, indicator=True, how='outer')
no_rookies = no_rookies[no_rookies['_merge'] == 'left_only']

# combined_stats_no_rookies = pd.merge(combined_stats,combined_rookies, on=['Player','Year'], how='left')

In [37]:
no_rookies.shape

(4890, 51)

In [38]:
del no_rookies['_merge']

# Add Player stats into our Salary list

Now we need to extract needed player information and add it to our FA list to complete our cleaned data process. We have to determine what we need and how we are going to do it. For now we want to look at data from the season previous to when the player becomes a free agent.

But first, let's save the data.

In [39]:
def save_dataset(data,filename):
    with open(filename, 'wb') as w:
        pickle.dump(data,w)

In [40]:
save_dataset(no_rookies,"../data/interim/stats.pickle")
save_dataset(salaries, "../data/interim/salaries.pickle")

Now we want to filter out the data for all the FA's on our list, but before we do that, we must look at one thing: the way naming is done differently between the salaries list (from ESPN.com) and the stats list (from Basketball-Reference.com). I noticed two weird things:

1. Suffixes are missing (notably Jr.) in Basketball-Reference list
2. Also players that go by initials (i.e. J.J. Redick in Basketball-Reference) are missing periods in the other list (i.e. JJ Redick in ESPN).

In [41]:
salaries['Player'] = salaries['Player'].map(lambda x: x.replace(' Jr.',""))

In [42]:
no_rookies['Player'] = no_rookies['Player'].map(lambda x: x.replace('.',""))

In [43]:
data_all = pd.merge(salaries,no_rookies, on=['Player','Year'], how='left')

In [44]:
data_all.columns

Index(['Player', 'Tm_x', 'Salary', 'Year', 'Pos_x', 'Pos_y', 'Age', 'Tm_y',
       'G', 'GS', 'MP_x', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [45]:
salaries.shape

(4511, 5)

In [46]:
data_all.shape

(4512, 53)

In [47]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4512 entries, 0 to 4511
Data columns (total 53 columns):
Player    4512 non-null object
Tm_x      4512 non-null object
Salary    4512 non-null int64
Year      4512 non-null int64
Pos_x     4512 non-null object
Pos_y     2906 non-null object
Age       2906 non-null float64
Tm_y      2906 non-null object
G         2906 non-null float64
GS        2906 non-null float64
MP_x      2906 non-null float64
FG        2906 non-null float64
FGA       2906 non-null float64
FG%       2904 non-null float64
3P        2906 non-null float64
3PA       2906 non-null float64
3P%       2610 non-null float64
2P        2906 non-null float64
2PA       2906 non-null float64
2P%       2899 non-null float64
eFG%      2904 non-null float64
FT        2906 non-null float64
FTA       2906 non-null float64
FT%       2875 non-null float64
ORB       2906 non-null float64
DRB       2906 non-null float64
TRB       2906 non-null float64
AST       2906 non-null float64
STL   

Missing alot of data; what is going on?

## Some column conflicts happened on the merge:

- Tm: I will use Tm_x as that was from the salary data. It is the team that paid the player on the season following (2018 salary is for the 2018-19 season, while 2018 stats is for the 2017-18 season)
- Pos: I will also use Pos_x although position ambiguity is really on the data collector's hands; some players can be either guard position, or either forward position, or some could be SG/SF. There is no real definition on positions now, as NBA is becoming more positionless and a guard is able to do what forwards used to, and vice versa. Even some centers handle the ball like a guard!

In [48]:
del data_all['Tm_y']
del data_all['Pos_y']


In [49]:
data_all.rename(columns={'Tm_x': 'Tm','Pos_x':'Pos','MP_x':'MP'}, inplace=True)

## Missing Data:

Here are the possibilities based on some research:

1. Some did not have stats because they were out of the NBA (not playing basketball entirely or overseas). These players should also be removed from the considerations.

2. Some players are missing a couple stats only (way to treat those datapoints will be explored in the next section)

In [50]:
data_all[(data_all.isnull().any(axis=1))]

,Player,Tm,Salary,Year,Pos,Age,G,GS,MP,FG,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
4,Gordon Hayward,Boston Celtics,31214295,2018,SF,27.0,1.0,1.0,5.0,1.0,...,0.0,17.9,0.0,0.0,0.0,0.056,-5.9,-6.1,-12.0,0.0
30,Rudy Gobert,Utah Jazz,23241573,2018,C,25.0,56.0,56.0,32.4,4.9,...,15.7,16.9,4.2,3.9,8.1,0.214,-0.3,4.6,4.3,2.9
32,DeAndre Jordan,New York Knicks,22897200,2018,C,29.0,77.0,77.0,31.5,4.8,...,16.1,15.2,6.0,3.4,9.4,0.186,0.5,1.6,2.1,2.5
54,Tristan Thompson,Cleveland Cavaliers,17469565,2018,C,26.0,53.0,22.0,20.2,2.5,...,11.8,12.6,1.8,0.7,2.5,0.113,-1.6,-0.9,-2.5,-0.1
75,Brandon Knight,Houston Rockets,14631250,2018,PG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,Marcin Gortat,LA Clippers,13565218,2018,C,33.0,82.0,82.0,25.3,3.5,...,13.5,15.5,2.4,2.4,4.9,0.113,-1.6,1.9,0.3,1.2
91,Miles Plumlee,Atlanta Hawks,12500000,2018,C,29.0,55.0,35.0,16.7,1.9,...,23.4,12.8,0.1,0.7,0.9,0.046,-3.8,1.3,-2.5,-0.1
100,T.J. Warren,Phoenix Suns,11750000,2018,SF,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,Omer Asik,Chicago Bulls,11286516,2018,C,31.0,18.0,0.0,10.1,0.5,...,24.5,8.7,-0.2,0.2,-0.1,-0.021,-8.1,0.0,-8.1,-0.3
127,Bogdan Bogdanovic,Sacramento Kings,9000000,2018,SG,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
playerinfo =['Player','Tm','Salary','Year','Pos']
rest = data_all.columns.difference(playerinfo)

In [52]:
played = data_all.dropna(thresh=20)

In [53]:
played.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2906 entries, 0 to 4502
Data columns (total 51 columns):
Player    2906 non-null object
Tm        2906 non-null object
Salary    2906 non-null int64
Year      2906 non-null int64
Pos       2906 non-null object
Age       2906 non-null float64
G         2906 non-null float64
GS        2906 non-null float64
MP        2906 non-null float64
FG        2906 non-null float64
FGA       2906 non-null float64
FG%       2904 non-null float64
3P        2906 non-null float64
3PA       2906 non-null float64
3P%       2610 non-null float64
2P        2906 non-null float64
2PA       2906 non-null float64
2P%       2899 non-null float64
eFG%      2904 non-null float64
FT        2906 non-null float64
FTA       2906 non-null float64
FT%       2875 non-null float64
ORB       2906 non-null float64
DRB       2906 non-null float64
TRB       2906 non-null float64
AST       2906 non-null float64
STL       2906 non-null float64
BLK       2906 non-null float64
TOV 

Save the data and move onto EDA:

In [54]:
FA=pickle.load(open("../data/raw/freeagents.pickle", "rb"))

In [55]:
FA[2018].head()

,Player (108 of 189),Pos.,Age,Type,From,To,Yrs,Dollars,Avg. Salary,2018 Cap Hit
0,Chris Paul,PG,33.0,UFA,HOU,HOU,4,"$159,730,592","$39,932,648",NaN
1,LeBron James,SF,33.0,UFA,CLE,LAL,4,"$153,312,846","$38,328,212","$35,654,150"
2,Nikola Jokic,C,23.0,RFA,DEN,DEN,5,"$147,710,050","$29,542,010",NaN
3,Paul George,SF,28.0,UFA,OKC,OKC,4,"$136,911,936","$34,227,984",NaN
4,Clint Capela,C,24.0,RFA,HOU,HOU,5,"$90,000,000","$18,000,000",NaN


In [56]:
FAS={}
for k,v in FA.items():
    v.columns=[re.sub(r"Player.+","Player",col) for col in v.columns]
    v.columns=[re.sub(r"\d+ Cap Hit","Cap Hit",col) for col in v.columns]
    v["Year"] = k
    FAS[k]=v

In [57]:
FAS[2018].head(5)

,Player,Pos.,Age,Type,From,To,Yrs,Dollars,Avg. Salary,Cap Hit,Year
0,Chris Paul,PG,33.0,UFA,HOU,HOU,4,"$159,730,592","$39,932,648",NaN,2018
1,LeBron James,SF,33.0,UFA,CLE,LAL,4,"$153,312,846","$38,328,212","$35,654,150",2018
2,Nikola Jokic,C,23.0,RFA,DEN,DEN,5,"$147,710,050","$29,542,010",NaN,2018
3,Paul George,SF,28.0,UFA,OKC,OKC,4,"$136,911,936","$34,227,984",NaN,2018
4,Clint Capela,C,24.0,RFA,HOU,HOU,5,"$90,000,000","$18,000,000",NaN,2018


In [58]:
freeagents = reduce(lambda x,y:pd.concat([x,y]),[v for k,v in FAS.items() if k != 2019])

In [59]:
freeagents.head(5)

,Player,Pos.,Age,Type,From,To,Yrs,Dollars,Avg. Salary,Cap Hit,Year
0,Chris Paul,PG,33.0,UFA,HOU,HOU,4,"$159,730,592","$39,932,648",NaN,2018
1,LeBron James,SF,33.0,UFA,CLE,LAL,4,"$153,312,846","$38,328,212","$35,654,150",2018
2,Nikola Jokic,C,23.0,RFA,DEN,DEN,5,"$147,710,050","$29,542,010",NaN,2018
3,Paul George,SF,28.0,UFA,OKC,OKC,4,"$136,911,936","$34,227,984",NaN,2018
4,Clint Capela,C,24.0,RFA,HOU,HOU,5,"$90,000,000","$18,000,000",NaN,2018


In [60]:
freeagents = freeagents[['Player','Year']]

In [61]:
FA_check = played.merge(freeagents, indicator=True, how='left')

In [77]:
played["FA"] = FA_check["_merge"]
played["FA"] = played["FA"].str.replace("left_only",'No').replace("both","Yes")

/Users/youngjeong/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/youngjeong/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [78]:
played

,Player,Tm,Salary,Year,Pos,Age,G,GS,MP,FG,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,FA
0,Stephen Curry,Golden State Warriors,37457154,2018,PG,29.0,51.0,51.0,32.0,8.4,...,31.0,7.2,1.9,9.1,0.267,9.9,-1.3,8.6,4.4,No
1,Russell Westbrook,Oklahoma City Thunder,35654150,2018,PG,29.0,80.0,80.0,36.4,9.5,...,34.1,5.5,4.5,10.1,0.166,5.1,3.1,8.2,7.5,No
2,Chris Paul,Houston Rockets,35654150,2018,PG,32.0,58.0,58.0,31.8,6.3,...,24.5,7.5,2.7,10.2,0.265,6.2,0.9,7.1,4.3,Yes
3,Blake Griffin,Detroit Pistons,32088932,2018,PF,28.0,58.0,58.0,34.0,7.5,...,28.9,3.2,1.8,4.9,0.120,2.9,-0.1,2.8,2.4,No
4,Gordon Hayward,Boston Celtics,31214295,2018,SF,27.0,1.0,1.0,5.0,1.0,...,17.9,0.0,0.0,0.0,0.056,-5.9,-6.1,-12.0,0.0,No
5,Kyle Lowry,Toronto Raptors,31200000,2018,PG,31.0,78.0,78.0,32.2,5.2,...,21.7,7.0,3.1,10.2,0.194,5.4,0.5,5.9,5.0,No
6,Paul George,Oklahoma City Thunder,30560700,2018,SF,27.0,79.0,79.0,36.6,7.3,...,25.7,5.0,3.9,8.9,0.147,2.5,0.0,2.6,3.3,Yes
7,Mike Conley,Memphis Grizzlies,30521115,2018,PG,30.0,12.0,12.0,31.1,5.3,...,25.9,0.4,0.2,0.6,0.077,1.2,-3.5,-2.3,0.0,No
8,James Harden,Houston Rockets,30421854,2018,PG,28.0,72.0,72.0,35.4,9.0,...,36.1,11.6,3.8,15.4,0.289,9.6,1.3,10.9,8.3,No
9,Kevin Durant,Golden State Warriors,30000000,2018,SF,29.0,68.0,68.0,34.2,9.3,...,30.4,7.5,2.9,10.4,0.215,5.0,0.7,5.6,4.5,Yes


In [79]:
save_dataset(played,"../data/processed/dataset2.pickle")